In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

HOST_IP = os.environ['HOST_IP']
DATABASE_USER = os.environ['DATABASE_USER']
DATABASE_PASSWORD = os.environ['DATABASE_PASSWORD']
DATABASE_PORT = os.environ['DATABASE_PORT']

connection_url = URL.create(
    "postgresql+psycopg2",
    username=DATABASE_USER,
    password=DATABASE_PASSWORD,
    host=HOST_IP,
    port=DATABASE_PORT,
    database="mimiciv"
)

engine = create_engine(connection_url)

In [ ]:
import pandas as pd
from sqlalchemy import text

query = text("SELECT * FROM mimiciv.mimiciv_hosp.admissions LIMIT 10")
pd.read_sql_query(query, engine)

In [ ]:
import json

query = text("""
SELECT json_build_object(
    n.nspname, 
    json_agg(
        json_build_object(
            t.tablename, 
            t.columns
        )
    )
)
FROM pg_catalog.pg_namespace n
JOIN (
    SELECT 
        t.schemaname, 
        t.tablename, 
        array_agg(c.column_name) as columns
    FROM pg_catalog.pg_tables t
    JOIN information_schema.columns c ON t.tablename = c.table_name AND t.schemaname = c.table_schema
    WHERE t.schemaname NOT IN ('information_schema', 'pg_catalog', 'pg_toast')
    GROUP BY t.schemaname, t.tablename
) t ON n.nspname = t.schemaname
GROUP BY n.nspname
""")

# Execute the query
with engine.connect() as con:
    result = con.execute(query).fetchall()

In [ ]:
# Formatting and printing the result
list_of_schemas = [res[0] for res in result]
formatted_result_1 = {k: v for d in list_of_schemas for k, v in d.items()}
formatted_result = {}
for outer_key, list_of_dicts in formatted_result_1.items():
    combined_dict = {}
    for inner_dict in list_of_dicts:
        combined_dict.update(inner_dict)
    formatted_result[outer_key] = combined_dict
formatted_result

In [ ]:
subject_id = 10000032
hadm_id = 29079034

query = text("select * from mimiciv.mimiciv_hosp.admissions where subject_id = :subject and hadm_id = :hadm;")
query = query.bindparams(subject=subject_id, hadm=hadm_id)

df = pd.read_sql_query(query, engine)

df

In [ ]:
# clone df
hospital_stay = df.copy()

for k, v in formatted_result['mimiciv_hosp'].items():
    if 'hadm_id' in v:
        if k == 'admissions':
            continue
        
        print(k)
        sql_query = text(f"select * from mimiciv.mimiciv_hosp.{k} where hadm_id = :hadm_id").bindparams(hadm_id=hadm_id)
        sql_df = pd.read_sql(sql_query, engine).drop('subject_id', axis=1, errors='ignore')
        
        # Prefix column with table name
        sql_df = sql_df.rename(columns=lambda x: f"{k}_{x}" if x != 'hadm_id' else x)
        
        # if more than one row, convert to list of dictionaries
        if len(sql_df) > 1:
            sql_df = sql_df.drop('hadm_id', axis=1, errors='ignore')
            sql_df = sql_df.to_dict('records')
            
            # add to df2 as a single cell in new column
            hospital_stay[k] = [sql_df]
        else:
            hospital_stay = hospital_stay.merge(sql_df, on='hadm_id', how='outer')

In [ ]:
hospital_stay

In [ ]:
ed_stay_query = text("select * from mimiciv.mimiciv_ed.edstays where hadm_id = :hadm_id").bindparams(hadm_id=hadm_id)

ed_stay_df = pd.read_sql(ed_stay_query, engine)

ed_stays = []

for row in ed_stay_df.iterrows():
    row_info = pd.DataFrame(row[1]).transpose()
    stay_id = row_info['stay_id'].values[0]
    
    for k, v in formatted_result['mimiciv_ed'].items():
        if 'stay_id' in v:
            if k == 'edstays':
                continue
            
            print(k)
            sql_query = text(f"select * from mimiciv.mimiciv_ed.{k} where stay_id = :stay_id").bindparams(stay_id=stay_id)
            sql_df = pd.read_sql(sql_query, engine).drop('subject_id', axis=1, errors='ignore')
            
            # Prefix column with table name
            sql_df = sql_df.rename(columns=lambda x: f"{k}_{x}" if x != 'stay_id' else x)
            
            # if more than one row, convert to list of dictionaries
            if len(sql_df) > 1:
                sql_df = sql_df.drop('stay_id', axis=1, errors='ignore')
                sql_df = sql_df.to_dict('records')
                                
                # add to df2 as a single cell in new column
                row_info[k] = [sql_df]
            else:
                row_info = row_info.merge(sql_df, on='stay_id', how='outer')
                
    ed_stays.append(row_info)
    
ed_stays = pd.concat(ed_stays, ignore_index=True)

In [ ]:
ed_stays

In [ ]:
icu_stay_query = text("select * from mimiciv.mimiciv_icu.icustays where hadm_id = :hadm_id").bindparams(hadm_id=hadm_id)

icu_stay_df = pd.read_sql(icu_stay_query, engine)

icu_stays = []

for row in icu_stay_df.iterrows():
    row_info = pd.DataFrame(row[1]).transpose()
    stay_id = row_info['stay_id'].values[0]
    
    for k, v in formatted_result['mimiciv_icu'].items():
        if 'stay_id' in v:
            if k == 'icustays':
                continue
            
            print(k)
            sql_query = text(f"select * from mimiciv.mimiciv_icu.{k} where stay_id = :stay_id").bindparams(stay_id=stay_id)
            sql_df = pd.read_sql(sql_query, engine).drop('subject_id', axis=1, errors='ignore')
            
            # Prefix column with table name
            sql_df = sql_df.rename(columns=lambda x: f"{k}_{x}" if x != 'stay_id' else x)
            
            # if more than one row, convert to list of dictionaries
            if len(sql_df) > 1:
                sql_df = sql_df.drop('stay_id', axis=1, errors='ignore')
                sql_df = sql_df.to_dict('records')
                                
                # add to df2 as a single cell in new column
                row_info[k] = [sql_df]
            else:
                row_info = row_info.merge(sql_df, on='stay_id', how='outer')
                
    icu_stays.append(row_info)
    
icu_stays = pd.concat(icu_stays, ignore_index=True)

In [ ]:
icu_stays